In [1]:
import sys; sys.path.insert(0, '../../')

In [2]:
import numpy as np
import tensorflow as tf
import nengo_dl
from pathlib import Path
from models.snn.factory import build_snn
from models.snn.train import train_model, evaluate_model
from models.preprocessing import load_preprocessed

2025-12-17 22:08:09.435444: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-12-17 22:08:09.435486: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-12-17 22:08:11.529500: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-12-17 22:08:13.427637: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-12-17 22:08:13.427767: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: l

In [3]:
splits, class_names = load_preprocessed('../../data/processed/')
X_train, y_train = splits['train']
X_val, y_val = splits['val']
X_test, y_test = splits['test']

Loaded train: X=(9, 6144), y=(9, 10)
Loaded val: X=(3, 6144), y=(3, 10)
Loaded test: X=(2, 6144), y=(2, 10)
Loaded class names: ['benjamin', 'christian', 'felix', 'jonas', 'leon', 'mark', 'marvin', 'ohman', 'veronica', 'yannes']


In [4]:
# Hyperparameters
hidden = 100
syn = 0.01
epochs = 10
learning_rate = 1e-3
batch_size = 32
checkpoint_dir = Path('../../checkpoints/snn')

In [5]:
# Build the SNN architecture
print("Building SNN architecture...")
net, inp, p_out = build_snn(
    n_features=X_train.shape[1],
    n_classes=len(class_names),
    n_neurons_hidden=hidden,
    synapse=syn,
)
print(f"Network built with {X_train.shape[1]} input features and {len(class_names)} output classes")

Building SNN architecture...
Network built with 6144 input features and 10 output classes


In [6]:
# Train the SNN
print("Starting training...")
history, sim = train_model(
    net=net,
    inp=inp,
    p_out=p_out,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    epochs=epochs,
    batch_size=batch_size,
    learning_rate=learning_rate,
    checkpoint_dir=checkpoint_dir,
)

Starting training...
Creating simulator...


NetworkContextError: `configure_settings` must be called within a Network context (`with nengo.Network(): ...`)

In [ ]:
# Evaluate on test set
print("\nEvaluating on test set...")
test_acc = evaluate_model(sim, inp, p_out, X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Display training history
print(f"\nTraining Summary:")
print(f"  Final train accuracy: {history.history['p_out_accuracy'][-1]:.4f}")
print(f"  Final val accuracy: {history.history['val_p_out_accuracy'][-1]:.4f}")
print(f"  Test accuracy: {test_acc:.4f}")

sim.close()